In [133]:
import numpy as np
import pandas as pd
import re
from scipy.stats import mode

In [134]:
train = pd.read_csv('~/datasets/titanic/train.csv')
test = pd.read_csv('~/datasets/titanic/test.csv')
y_train = train.pop('Survived')
train['source']='train'
test['source']='test'
df = pd.concat([train, test],ignore_index=True)
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,source
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


In [135]:
df.apply(lambda x: sum(x.isnull()))

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
source            0
dtype: int64

In [136]:
df = df.drop(['PassengerId','Ticket'], axis=1)
df.tail(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,source
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,8.0500,NaN,S,test
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C105,C,test
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,NaN,S,test
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,8.0500,NaN,S,test
1308,3,"Peter, Master. Michael J",male,NaN,1,1,22.3583,NaN,C,test


In [137]:
df.Age.fillna(df.Age.mean(), inplace = True) 
df.Fare.fillna(df.Fare.mean(), inplace = True)
df.Cabin.fillna('U', inplace = True)
df.Embarked.fillna(mode(df.Embarked).mode[0], inplace = True)
df.apply(lambda x: sum(x.isnull()))

/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
source      0
dtype: int64

In [138]:
df.tail(5)

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,source
1304,3,"Spector, Mr. Woolf",male,29.881138,0,0,8.0500,U,S,test
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,108.9000,C105,C,test
1306,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,7.2500,U,S,test
1307,3,"Ware, Mr. Frederick",male,29.881138,0,0,8.0500,U,S,test
1308,3,"Peter, Master. Michael J",male,29.881138,1,1,22.3583,U,C,test


In [139]:
df['Deck'] = df['Cabin'].apply(lambda x: x[0])
df = df.drop('Cabin',axis=1)

In [140]:
df.Deck.value_counts()

U    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Deck, dtype: int64

In [141]:
def AClass(x):
    if x < 19.0:
        return 1
    elif x >= 19.0 and x < 25.0:
        return 2
    elif x >= 25.0 and x < 29.881:
        return 3
    elif x >= 29.881 and x < 30.5:
        return 4
    elif x >= 30.4 and x < 40.1:
        return 5
    else:
        return 6

df['Age_Category'] = df['Age'].apply(lambda x: AClass(x)) 
df = df.drop('Age', axis=1)

In [142]:
def FClass(x):
    if x < 7.775:
        return 1
    elif x >= 7.775 and x < 8.662:
        return 2
    elif x >= 8.662 and x < 14.454:
        return 3
    elif x >= 14.454 and x < 26.0:
        return 4
    elif x >= 26.0 and x < 53.1:
        return 5
    else:
        return 6

df['Fare_Category'] = df['Fare'].apply(lambda x: FClass(x)) 
df = df.drop('Fare', axis=1)

In [143]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [144]:
def IAClass(x):
    if x > 1:
        return 0
    else:
        return 1

df['IsAlone'] = df['FamilySize'].apply(lambda x: IAClass(x)) 
df = df.drop(['SibSp','Parch','FamilySize'], axis=1)

In [145]:
df['Title'] = df['Name'].str.extract(r',\s(.*)\.', expand=False)
Title_Dictionary = {
                    "Capt":       "Officer",
                    "Col":        "Officer",
                    "Major":      "Officer",
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Dr":         "Officer",
                    "Rev":        "Officer",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Mme":        "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Mrs",
                    "Mr" :        "Mr",
                    "Mrs" :       "Mrs",
                    "Miss" :      "Miss",
                    "Master" :    "Master",
                    "Lady" :      "Royalty"

                    }


df['Title'] = df['Title'].apply(lambda x: df.Title.map(Title_Dictionary))
df = df.drop('Name', axis=1)

In [146]:
df.Sex = df.Sex.map({'male':1,'female':0})

In [147]:
df.tail(5)

,Pclass,Sex,Embarked,source,Deck,Age_Category,Fare_Category,IsAlone,Title
1304,3,1,S,test,U,4,2,1,Mr
1305,1,0,C,test,C,5,6,1,Mr
1306,3,1,S,test,U,5,1,1,Mr
1307,3,1,S,test,U,4,2,1,Mr
1308,3,1,C,test,U,4,4,0,Mr


In [148]:
categorical_columns = [x for x in df.dtypes.index if df.dtypes[x]=='object']
categorical_columns

['Embarked', 'source', 'Deck', 'Title']

In [149]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Embarked','Title','Deck']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])
df = pd.get_dummies(df, columns=['Pclass','Embarked','Title','Deck','Age_Category','Fare_Category'])
df.head()

,Sex,source,IsAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_0,Embarked_1,Embarked_2,Title_0,...,Age_Category_3,Age_Category_4,Age_Category_5,Age_Category_6,Fare_Category_1,Fare_Category_2,Fare_Category_3,Fare_Category_4,Fare_Category_5,Fare_Category_6
0,1,train,0,0,0,1,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,0,train,0,1,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2,0,train,1,0,0,1,0,0,1,1,...,1,0,0,0,0,1,0,0,0,0
3,0,train,0,1,0,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,1
4,1,train,1,0,0,1,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0


In [150]:
train = df.loc[df['source']=="train"]
test = df.loc[df['source']=="test"]

train = train.drop('source',axis=1)
test = test.drop('source',axis=1)

X_train = train.loc[:,:]

X_test = test.loc[:,:]

In [151]:
import catboost as cb
from sklearn.model_selection import GridSearchCV

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
model = cb.CatBoostClassifier()
grid_search = GridSearchCV(model, params, scoring="roc_auc", cv = 3)
grid_search.fit(X_train, y_train)
grid_search.best_estimator_

model = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
model.fit(X_train,y_train)


0:	learn: 0.6725191	total: 32.8ms	remaining: 9.82s
1:	learn: 0.6588348	total: 62.3ms	remaining: 9.29s
2:	learn: 0.6430167	total: 86.1ms	remaining: 8.53s
3:	learn: 0.6280303	total: 108ms	remaining: 8s
4:	learn: 0.6127050	total: 130ms	remaining: 7.67s
5:	learn: 0.5981140	total: 150ms	remaining: 7.33s
6:	learn: 0.5842850	total: 178ms	remaining: 7.47s
7:	learn: 0.5727968	total: 198ms	remaining: 7.22s
8:	learn: 0.5607220	total: 224ms	remaining: 7.23s
9:	learn: 0.5519479	total: 246ms	remaining: 7.14s
10:	learn: 0.5462823	total: 280ms	remaining: 7.36s
11:	learn: 0.5385964	total: 309ms	remaining: 7.43s
12:	learn: 0.5288627	total: 331ms	remaining: 7.31s
13:	learn: 0.5259701	total: 342ms	remaining: 6.98s
14:	learn: 0.5192275	total: 357ms	remaining: 6.79s
15:	learn: 0.5115054	total: 367ms	remaining: 6.51s
16:	learn: 0.5079730	total: 376ms	remaining: 6.26s
17:	learn: 0.5009344	total: 394ms	remaining: 6.17s
18:	learn: 0.4943626	total: 406ms	remaining: 6s
19:	learn: 0.4883962	total: 415ms	remaining:

163:	learn: 0.3621451	total: 3.68s	remaining: 3.05s
164:	learn: 0.3619806	total: 3.71s	remaining: 3.04s
165:	learn: 0.3619023	total: 3.72s	remaining: 3s
166:	learn: 0.3618816	total: 3.73s	remaining: 2.97s
167:	learn: 0.3615965	total: 3.75s	remaining: 2.95s
168:	learn: 0.3605137	total: 3.77s	remaining: 2.92s
169:	learn: 0.3604309	total: 3.78s	remaining: 2.89s
170:	learn: 0.3604239	total: 3.79s	remaining: 2.85s
171:	learn: 0.3601614	total: 3.79s	remaining: 2.82s
172:	learn: 0.3601571	total: 3.8s	remaining: 2.79s
173:	learn: 0.3598473	total: 3.81s	remaining: 2.76s
174:	learn: 0.3597439	total: 3.82s	remaining: 2.73s
175:	learn: 0.3596266	total: 3.83s	remaining: 2.7s
176:	learn: 0.3590870	total: 3.84s	remaining: 2.67s
177:	learn: 0.3590379	total: 3.85s	remaining: 2.64s
178:	learn: 0.3589863	total: 3.86s	remaining: 2.61s
179:	learn: 0.3588946	total: 3.87s	remaining: 2.58s
180:	learn: 0.3588205	total: 3.88s	remaining: 2.55s
181:	learn: 0.3585061	total: 3.9s	remaining: 2.53s
182:	learn: 0.3583

22:	learn: 0.5025196	total: 262ms	remaining: 3.15s
23:	learn: 0.4979821	total: 287ms	remaining: 3.3s
24:	learn: 0.4941597	total: 306ms	remaining: 3.37s
25:	learn: 0.4905328	total: 325ms	remaining: 3.42s
26:	learn: 0.4880609	total: 341ms	remaining: 3.45s
27:	learn: 0.4872636	total: 358ms	remaining: 3.48s
28:	learn: 0.4855396	total: 374ms	remaining: 3.49s
29:	learn: 0.4845555	total: 390ms	remaining: 3.51s
30:	learn: 0.4838774	total: 406ms	remaining: 3.53s
31:	learn: 0.4819823	total: 422ms	remaining: 3.53s
32:	learn: 0.4802551	total: 440ms	remaining: 3.56s
33:	learn: 0.4776613	total: 461ms	remaining: 3.61s
34:	learn: 0.4756921	total: 479ms	remaining: 3.63s
35:	learn: 0.4732277	total: 494ms	remaining: 3.62s
36:	learn: 0.4713550	total: 504ms	remaining: 3.58s
37:	learn: 0.4707155	total: 516ms	remaining: 3.56s
38:	learn: 0.4698178	total: 538ms	remaining: 3.6s
39:	learn: 0.4686883	total: 570ms	remaining: 3.7s
40:	learn: 0.4647061	total: 600ms	remaining: 3.79s
41:	learn: 0.4614556	total: 628ms	

182:	learn: 0.3823178	total: 2.6s	remaining: 1.66s
183:	learn: 0.3820200	total: 2.61s	remaining: 1.65s
184:	learn: 0.3816901	total: 2.63s	remaining: 1.64s
185:	learn: 0.3816643	total: 2.64s	remaining: 1.62s
186:	learn: 0.3816640	total: 2.65s	remaining: 1.6s
187:	learn: 0.3815756	total: 2.67s	remaining: 1.59s
188:	learn: 0.3815708	total: 2.68s	remaining: 1.57s
189:	learn: 0.3815592	total: 2.69s	remaining: 1.55s
190:	learn: 0.3814409	total: 2.7s	remaining: 1.54s
191:	learn: 0.3808833	total: 2.7s	remaining: 1.52s
192:	learn: 0.3805601	total: 2.71s	remaining: 1.5s
193:	learn: 0.3804032	total: 2.72s	remaining: 1.49s
194:	learn: 0.3800636	total: 2.76s	remaining: 1.48s
195:	learn: 0.3800534	total: 2.77s	remaining: 1.47s
196:	learn: 0.3800316	total: 2.79s	remaining: 1.46s
197:	learn: 0.3798827	total: 2.81s	remaining: 1.45s
198:	learn: 0.3795792	total: 2.84s	remaining: 1.44s
199:	learn: 0.3794090	total: 2.87s	remaining: 1.43s
200:	learn: 0.3787719	total: 2.88s	remaining: 1.42s
201:	learn: 0.378

42:	learn: 0.4549770	total: 693ms	remaining: 4.14s
43:	learn: 0.4534798	total: 712ms	remaining: 4.14s
44:	learn: 0.4527433	total: 725ms	remaining: 4.11s
45:	learn: 0.4513262	total: 736ms	remaining: 4.07s
46:	learn: 0.4508149	total: 745ms	remaining: 4.01s
47:	learn: 0.4494866	total: 756ms	remaining: 3.97s
48:	learn: 0.4489957	total: 765ms	remaining: 3.92s
49:	learn: 0.4474935	total: 775ms	remaining: 3.87s
50:	learn: 0.4458739	total: 786ms	remaining: 3.83s
51:	learn: 0.4453637	total: 795ms	remaining: 3.79s
52:	learn: 0.4437019	total: 805ms	remaining: 3.75s
53:	learn: 0.4425132	total: 816ms	remaining: 3.72s
54:	learn: 0.4421184	total: 838ms	remaining: 3.73s
55:	learn: 0.4410205	total: 850ms	remaining: 3.7s
56:	learn: 0.4404662	total: 875ms	remaining: 3.73s
57:	learn: 0.4394525	total: 887ms	remaining: 3.7s
58:	learn: 0.4390882	total: 916ms	remaining: 3.74s
59:	learn: 0.4374366	total: 936ms	remaining: 3.74s
60:	learn: 0.4361360	total: 945ms	remaining: 3.7s
61:	learn: 0.4358670	total: 967ms	

...

154:	learn: 0.3168321	total: 2.33s	remaining: 2.18s
155:	learn: 0.3168276	total: 2.34s	remaining: 2.16s
156:	learn: 0.3167943	total: 2.35s	remaining: 2.14s
157:	learn: 0.3163195	total: 2.37s	remaining: 2.13s
158:	learn: 0.3163162	total: 2.38s	remaining: 2.11s
159:	learn: 0.3162708	total: 2.39s	remaining: 2.09s
160:	learn: 0.3162495	total: 2.4s	remaining: 2.07s
161:	learn: 0.3162180	total: 2.41s	remaining: 2.05s
162:	learn: 0.3141220	total: 2.48s	remaining: 2.08s
163:	learn: 0.3141207	total: 2.49s	remaining: 2.07s
164:	learn: 0.3140142	total: 2.51s	remaining: 2.05s
165:	learn: 0.3139838	total: 2.52s	remaining: 2.04s
166:	learn: 0.3133130	total: 2.55s	remaining: 2.03s
167:	learn: 0.3132732	total: 2.57s	remaining: 2.02s
168:	learn: 0.3132727	total: 2.58s	remaining: 2s
169:	learn: 0.3132686	total: 2.59s	remaining: 1.98s
170:	learn: 0.3132592	total: 2.61s	remaining: 1.97s
171:	learn: 0.3132310	total: 2.62s	remaining: 1.95s
172:	learn: 0.3132063	total: 2.64s	remaining: 1.94s
173:	learn: 0.31

14:	learn: 0.5172731	total: 373ms	remaining: 7.09s
15:	learn: 0.5106096	total: 385ms	remaining: 6.83s
16:	learn: 0.5035288	total: 396ms	remaining: 6.6s
17:	learn: 0.4972274	total: 412ms	remaining: 6.45s
18:	learn: 0.4915814	total: 423ms	remaining: 6.26s
19:	learn: 0.4885690	total: 432ms	remaining: 6.05s
20:	learn: 0.4845569	total: 442ms	remaining: 5.88s
21:	learn: 0.4805354	total: 452ms	remaining: 5.72s
22:	learn: 0.4775237	total: 461ms	remaining: 5.55s
23:	learn: 0.4724544	total: 473ms	remaining: 5.44s
24:	learn: 0.4699635	total: 483ms	remaining: 5.32s
25:	learn: 0.4674960	total: 492ms	remaining: 5.18s
26:	learn: 0.4648345	total: 503ms	remaining: 5.09s
27:	learn: 0.4618499	total: 514ms	remaining: 5s
28:	learn: 0.4583391	total: 525ms	remaining: 4.9s
29:	learn: 0.4551974	total: 538ms	remaining: 4.84s
30:	learn: 0.4532022	total: 548ms	remaining: 4.76s
31:	learn: 0.4503406	total: 560ms	remaining: 4.69s
32:	learn: 0.4496576	total: 574ms	remaining: 4.64s
33:	learn: 0.4477348	total: 585ms	re

179:	learn: 0.3567441	total: 2.82s	remaining: 1.88s
180:	learn: 0.3565908	total: 2.83s	remaining: 1.86s
181:	learn: 0.3564077	total: 2.84s	remaining: 1.84s
182:	learn: 0.3563270	total: 2.85s	remaining: 1.82s
183:	learn: 0.3562821	total: 2.86s	remaining: 1.8s
184:	learn: 0.3561381	total: 2.87s	remaining: 1.78s
185:	learn: 0.3560525	total: 2.88s	remaining: 1.76s
186:	learn: 0.3560141	total: 2.89s	remaining: 1.75s
187:	learn: 0.3560102	total: 2.9s	remaining: 1.73s
188:	learn: 0.3559306	total: 2.91s	remaining: 1.71s
189:	learn: 0.3558516	total: 2.92s	remaining: 1.69s
190:	learn: 0.3558294	total: 2.92s	remaining: 1.67s
191:	learn: 0.3555938	total: 2.94s	remaining: 1.65s
192:	learn: 0.3555593	total: 2.95s	remaining: 1.64s
193:	learn: 0.3555347	total: 2.96s	remaining: 1.62s
194:	learn: 0.3555185	total: 2.97s	remaining: 1.6s
195:	learn: 0.3555129	total: 2.98s	remaining: 1.58s
196:	learn: 0.3554625	total: 2.99s	remaining: 1.56s
197:	learn: 0.3554417	total: 3s	remaining: 1.54s
198:	learn: 0.3554

65:	total: 906ms	remaining: 5.96s
66:	total: 919ms	remaining: 5.94s
67:	total: 928ms	remaining: 5.89s
68:	total: 938ms	remaining: 5.86s
69:	total: 952ms	remaining: 5.85s
70:	total: 960ms	remaining: 5.8s
71:	total: 971ms	remaining: 5.77s
72:	total: 983ms	remaining: 5.75s
73:	total: 1.02s	remaining: 5.86s
74:	total: 1.03s	remaining: 5.83s
75:	total: 1.04s	remaining: 5.79s
76:	total: 1.05s	remaining: 5.78s
77:	total: 1.06s	remaining: 5.75s
78:	total: 1.07s	remaining: 5.72s
79:	total: 1.08s	remaining: 5.7s
80:	total: 1.1s	remaining: 5.67s
81:	total: 1.11s	remaining: 5.65s
82:	total: 1.12s	remaining: 5.62s
83:	total: 1.13s	remaining: 5.61s
84:	total: 1.14s	remaining: 5.59s
85:	total: 1.15s	remaining: 5.56s
86:	total: 1.16s	remaining: 5.53s
87:	total: 1.17s	remaining: 5.5s
88:	total: 1.18s	remaining: 5.47s
89:	total: 1.2s	remaining: 5.47s
90:	total: 1.21s	remaining: 5.44s
91:	total: 1.22s	remaining: 5.41s
92:	total: 1.23s	remaining: 5.38s
93:	total: 1.24s	remaining: 5.35s
94:	total: 1.25s	re

309:	total: 4.04s	remaining: 2.47s
310:	total: 4.05s	remaining: 2.46s
311:	total: 4.05s	remaining: 2.44s
312:	total: 4.07s	remaining: 2.43s
313:	total: 4.08s	remaining: 2.41s
314:	total: 4.08s	remaining: 2.4s
315:	total: 4.09s	remaining: 2.38s
316:	total: 4.11s	remaining: 2.37s
317:	total: 4.12s	remaining: 2.35s
318:	total: 4.12s	remaining: 2.34s
319:	total: 4.13s	remaining: 2.33s
320:	total: 4.15s	remaining: 2.31s
321:	total: 4.16s	remaining: 2.3s
322:	total: 4.17s	remaining: 2.29s
323:	total: 4.18s	remaining: 2.27s
324:	total: 4.19s	remaining: 2.26s
325:	total: 4.2s	remaining: 2.24s
326:	total: 4.21s	remaining: 2.23s
327:	total: 4.22s	remaining: 2.21s
328:	total: 4.23s	remaining: 2.2s
329:	total: 4.24s	remaining: 2.19s
330:	total: 4.25s	remaining: 2.17s
331:	total: 4.26s	remaining: 2.16s
332:	total: 4.27s	remaining: 2.14s
333:	total: 4.28s	remaining: 2.13s
334:	total: 4.29s	remaining: 2.11s
335:	total: 4.3s	remaining: 2.1s
336:	total: 4.31s	remaining: 2.08s
337:	total: 4.32s	remainin

In [152]:
y_pred = model.predict(X_test)

In [153]:
y_pred

array([0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [154]:
sample_file = pd.read_csv('~/datasets/titanic/gender_submission.csv')
sample_file['Survived'] = y_pred
sample_file.to_csv('titanic_submission1.csv', index=False)